In [1]:
import polars as pl
import calendar
import gc
import json
import glob
import hashlib

In [2]:
def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent = 2)

In [3]:
d_ss = j2d("../../share3/mat/d_ss.json")

In [4]:
d_ss

{'solt': '89sryf', 'stretch': 62}

In [5]:
l_ = j2d("../../share3/mat/drug_codes.json")['drugcode']
drgset = set(l_)
l_[:3]

['610463145', '621986101', '620871601']

In [6]:
lx = [str(i) for i in range(0, 10)]
lx

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [7]:
# f = "../../share3/temp/receipt_drug.parquet"
f = "../../share3/receipt_drug.parquet"
df_d = pl.scan_parquet(f,low_memory= True)\
    .filter(pl.col("tonpuku_flg") == "0")\
    .filter(pl.col("drug_code").is_in(drgset))\
    .with_columns(
            pl.col("shiyouryou").cast(float)
    )\
    .select(pl.col("receipt_ym","receipt_id","line_no","kojin_id", "drug_code","shiyouryou"))\
    .with_columns(
        pl.col("kojin_id").hash(seed=d_ss["stretch"]).cast(pl.String).str.slice(-1,1).alias("grp"),
    )

# df_d.fetch()

In [8]:
f_temp = "../../share3/receipt_drug_filterd.parquet"
# df_d.sink_parquet(f_temp,maintain_order=False)

In [9]:
%%time
# lx = df_d.select(pl.col("grp")).unique().collect(streaming=True)["grp"].to_list()
print(lx)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
CPU times: user 24 µs, sys: 10 µs, total: 34 µs
Wall time: 38.4 µs


In [12]:
%%time
df_d = pl.scan_parquet(f_temp,low_memory= True)
for k in lx:
    # print(k)
    f_out = "../../share3/drg/df_drg_split_{}.parquet".format(k)
    print(f_temp, f_out)
    df_d\
        .filter(pl.col("grp") == k)\
        .sink_parquet(f_out,maintain_order=False)

    gc.collect()

../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_0.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_1.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_2.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_3.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_4.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_5.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_6.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_7.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_8.parquet
../../share3/receipt_drug_filterd.parquet ../../share3/drg/df_drg_split_9.parquet
CPU times: user 39min 7s, sys: 1h 24min 25s, total: 2h 3min 33s
Wall time: 1h 40min 19s
